## AMSE M1 S2A 2022 Stochastic Processes in Economics
### Avinash TAMBY
### 16 February 2021

I have created a program which simulates many trajectories of a Markov chain and reports average time in each state and average time to absorption.
 
I have also generated code to calculate the theoretical values for:
Type of Markov Chain (regular, irreducible but not regular, absorbing, or none of the 3)
the stationary distribution (for regular chains)
the fundamental matrix for absorbing chains (and the theoretical probabilities of ending in each absorbing state and the theoretical time to absorption)

In [1]:
############################################################
# Title: AMSE M1 S2A Markov Chains in Python Midterm
# Author: Avinash TAMBY
# Date: 11 February 2022
############################################################

import numpy as np

############################################################
# Simulate a Markov Chain trajectory
############################################################

def simulation(x0, A, chain_len):
    prev_state = np.random.choice([a for a in range(len(x0))], p=x0) # randomly choose an initial state based on initial state distribution
    sim = [prev_state] # set initial state in simulated vector of responses
    time = []

    # for loop appends next state to vector of responses
    for i in range(chain_len):
        curr_state = np.random.choice(len(x0), p=A[prev_state])
        if prev_state not in np.where(A.diagonal() == 1)[0].tolist():
            time.append(True)
        sim.append(curr_state)
        prev_state=curr_state
        i+=1

    game_length = sum(time)
    
    return sim, game_length


############################################################
# Simulate many Markov Chain trajectories
############################################################

def simulated_trajectories(x0, A, chain_len, n_trajectories):
    test = [simulation(x0, A, 100)[0]]
    test2 = [simulation(x0, A, 100)[1]]

    for i in range(n_trajectories):
        test = np.append(test, ([simulation(x0, A, chain_len)[0]]), axis=0)
        test2 = np.append(test2, ([simulation(x0, A, chain_len)[1]]), axis=0)

    time = np.unique(test, return_counts=True)[1]/sum(np.unique(test, return_counts=True)[1])
    return "Simulated average time in each state: ", time.tolist(), "Simulated average time before absorption: ", np.average(test2)
    

############################################################
# Check if Markov Chain is irreducible
############################################################
def irreducible_check(A):
    x = []
    for k in range(len(A)):
        prev_state = k
        sim = [prev_state]
        for i in range(1000):
            curr_state = np.random.choice(len(A), p=A[prev_state])
            sim.append(curr_state)
            prev_state=curr_state
            i+=1
        if list(set(sim)) == [i for i in range(len(A))]: # Check if all states are accessed in a 1,000-length chain.
            x.append(True)
        else:
            x.append(False)
        k+=1
    if sum(x) == len(x): # If all states can be accessed in 1,000 steps from any initial state, the MC is irreducible.
        return True
    else:
        return False


############################################################
# Check if Markov Chain is regular
############################################################
def regular_check(A):
    if np.linalg.matrix_power(A, 100000).all(): 
        return True
    else:
        return False


############################################################
# Check if Markov Chain is absorbing
############################################################
def absorbing_check(A):
    if A.diagonal().any(): # if 1 or more diagonals ==1, check if last state in simulation is an absorbing state for all possible initial states
        x = []
        for k in range(len(A)):
            prev_state = k
            sim = [prev_state]
            for i in range(1000):
                curr_state = np.random.choice(len(A), p=A[prev_state])
                sim.append(curr_state)
                prev_state=curr_state
                i+=1
            if sim[-1] in np.where(A.diagonal() == 1)[0].tolist(): # Check if last element in simulation is an absorbing state
                x.append(True)
            else:
                x.append(False)
            k+=1
        if sum(x) == len(x): # If all initial states end in an absorbing state, the MC is absorbing.
            return True
        else:
            return False
    else: # if no diagonals == 1, not absorbing
        return False


############################################################
# Calculate the invariant distrubtion if MC is regular
############################################################
def invar_dist(A):
    if regular_check(A):
        return np.real(np.linalg.eig(A.T)[1][:, np.argmin(np.abs(np.linalg.eig(A.T)[0] - 1))]).T/np.sum(np.real(np.linalg.eig(A.T)[1][:, np.argmin(np.abs(np.linalg.eig(A.T)[0] - 1))]).T)
    else:
        return "Stationary distribution for non-regular MCs are not unique."


############################################################
# Calculate statistics of absorbing MCs
############################################################
def absorbing_markov(x0, A):
    if absorbing_check(A):
        n_abs_sts = len([i for i in A if 1 in i]) # Find number of absorbing states,
        Q = A[:len(A) - n_abs_sts,:len(A) - n_abs_sts] # Upper Left
        R = A[:len(A) - n_abs_sts,len(A) - n_abs_sts:] # Upper Right
        Z = A[len(A) - n_abs_sts:,:len(A) - n_abs_sts] # Lower Left
        I  = A[len(A) - n_abs_sts:,len(A) - n_abs_sts:] # Lower Right
        F = np.linalg.inv(np.identity(len(A) - n_abs_sts) - Q)
        U = np.concatenate((np.zeros((len(A) - n_abs_sts,len(A) - n_abs_sts)), np.matmul(F, R)), axis=1)
        L = np.concatenate((np.zeros((n_abs_sts, len(A) - n_abs_sts)), np.identity(n_abs_sts)), axis=1)
        P = np.concatenate((U, L), axis=0)
        return "Chain is absorbing. The absorbing states are", np.where(A.diagonal() == 1)[0].tolist(), "The theoretical long-term probabiilities of ending in each absorbing state are", np.matmul(np.transpose(P), x0).tolist(), ". The theoretical average time before absorption is", np.matmul(F, x0[:len(A) - n_abs_sts]).sum()
    else:
        return "Chain is not absorbing.",

############################################################
# Calculate statistics of Markov Chains
############################################################
def theoretical_statistics(x0, A):
    if regular_check(A):
        return "Chain is regular and its stationary distribution is", invar_dist(A).tolist(),
    elif irreducible_check(A) and regular_check(A) == False:
        return "Chain is irreducible but not regular."
    elif irreducible_check(A) == False and absorbing_check(A) == False:
        return "Chain is neither irreducible nor absorbing."
    elif absorbing_check(A):
        return absorbing_markov(x0, A)

#### Exercise 1
2 players *A* and *B* play tennis. We assume that all points are won by *A* with probability 3/5, and are independent. Initially players are tied and a game is won by a player as soon as he has won exactly 2 more points than his opponent
1. Model this game through an absorbing Markov chain with 5 states :
    E = {*tie(deuce)*, *Advantage*A, *Advantage*B, *Awins, andBwins*}. Explicit the transition matrix.

##### 1
$$
\begin{pmatrix}
tie \\ AdvA \\ AdvB \\ Awin \\ Bwin \\
\end{pmatrix}
\begin{pmatrix}
0 & \frac{3}{5} &\frac{2}{5} & 0 & 0 \\
\frac{2}{5} & 0 & 0 &\frac{3}{5} & 0 \\
\frac{3}{5} & 0 & 0 & 0 &\frac{2}{5} \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1 \\
\end{pmatrix}
$$

In [2]:
# Exercise 1
# 5 states: 0=tie; 1=AdvA; 2=AdvB; 3=Awins; 4=Bwins
initial_distribution = np.array([1, 0, 0, 0, 0]) # initial distribution
transition_matrix = np.array([[0, 0.6, 0.4, 0, 0], [0.4, 0, 0, .6, 0], [0.6, 0, 0, 0, 0.4], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]])
chain_length = 100
n_trajectories = 2000

print(simulated_trajectories(initial_distribution, transition_matrix, chain_length, n_trajectories))
print(theoretical_statistics(initial_distribution, transition_matrix))

('Simulated average time in each state: ', [0.0190201928738601, 0.011375500368627568, 0.007644692505232532, 0.6646577701248386, 0.2973018441274412], 'Simulated average time before absorption: ', 3.8590704647676164)
('Chain is absorbing. The absorbing states are', [3, 4], 'The theoretical long-term probabiilities of ending in each absorbing state are', [0.0, 0.0, 0.0, 0.6923076923076924, 0.3076923076923077], '. The theoretical average time before absorption is', 3.846153846153846)


As seen in the vector above, the theoretical long-term probabilities of ending in each absorbing state, $\begin{pmatrix}
0 & 0 & 0 & \frac{9}{13} & \frac{4}{13} \\ \end{pmatrix} or \begin{pmatrix}
0 & 0 & 0 & 0.6923076923076924, 0.3076923076923077 \\ \end{pmatrix}.$

The probability that A wins is $\frac{9}{13}$ or 0.6923076923076924.

The simulated time spent in state "A wins" (the average of 2000 trajectories of chain length 100), which represents the simulated probability that A wins is calculated above and is very close to the true theoretical value.

The expected number of points played, or the theoretical average time before absorption is 3.846153846153846. The simulated value is calculated above and is also very close.

#### Exercise 2
Consider the Markov chain on E = {1, 2, 3, 4}, with transition matrix
$$
\begin{pmatrix}
0 & \frac{1}{2} &\frac{1}{2} & 0 \\
\frac{1}{16} & \frac{7}{16} & 0 & \frac{1}{2} \\
\frac{1}{16} & 0 & \frac{7}{16} & \frac{1}{2} \\
0 & \frac{1}{4} &\frac{1}{4} & \frac{1}{2} \\\end{pmatrix}
$$

1. We see below the square of the transition matrix has strictly positive entries for every element. The Markov chain is thus regular. Regular chains are subsets of irreducible chains, to the Markov chain is also irreducible.

In [3]:
TM = np.array([[0, .5, .5, 0], [1/16, 7/16, 0, .5], [1/16, 0, 7/16, .5], [0, .25, .25, .5]])
np.linalg.matrix_power(TM, 2)

array([[0.0625    , 0.21875   , 0.21875   , 0.5       ],
       [0.02734375, 0.34765625, 0.15625   , 0.46875   ],
       [0.02734375, 0.15625   , 0.34765625, 0.46875   ],
       [0.03125   , 0.234375  , 0.234375  , 0.5       ]])

In [4]:
# Exercise 2
initial_distribution = np.array([.25, .25, .25, .25])
transition_matrix = np.array([[0, .5, .5, 0], [1/16, 7/16, 0, .5], [1/16, 0, 7/16, .5], [0, .25, .25, .5]])
chain_length = 100
n_trajectories = 2000

print(simulated_trajectories(initial_distribution, transition_matrix, chain_length, n_trajectories))
print(theoretical_statistics(initial_distribution, transition_matrix))

('Simulated average time in each state: ', [0.03226604519522417, 0.2435861277282151, 0.24170093171236165, 0.4824468953641991], 'Simulated average time before absorption: ', 100.0)
('Chain is regular and its stationary distribution is', [0.030303030303030314, 0.24242424242424224, 0.24242424242424251, 0.48484848484848486])


3. The invariant distribution, or the average time in each state, as seen in the output above is $\pi = \begin{pmatrix} 0.030303030303030314, 0.24242424242424224, 0.24242424242424251, 0.48484848484848486 \\\end{pmatrix}$ or $\pi = \begin{pmatrix} \frac{1}{33} & \frac{8}{33} &\frac{8}{33} & \frac{16}{33} \\\end{pmatrix}.$

The simulated invariant distribution, as calculated in the code output above is very close.

4. $\lim_{n \to \infty } \mathbb{P}_1(X_n= 3) and \lim_{n \to \infty } \mathbb{P}_4(X_n= 3)$ can be found from the invariant distribution. Both are equal to $\pi_3$ which is equal to $\frac{8}{33}$ or 0.24242424242424251.

The expected first return time in state 2 is $\frac{1}{\pi_2} = \frac{1}{\frac{8}{33}} = \frac{33}{8}.$

#### Exercise 3
1. The transition matrix is:
$$
\begin{pmatrix}
0 & 1 & 0 & 0 \\
\frac{1}{4} & \frac{1}{2} & \frac{1}{4} & 0 \\
0 & \frac{1}{2} & 0 & \frac{1}{2} \\
0 & 0 & 1 & 0 \\
\end{pmatrix}
$$

2. As seen in the calculation before, the 4th power of the transition matrix has all elements strictly > 0, meaning the Markov chain is regular, which by definition means it is also irreducible.

In [5]:
TM = np.array([[0, 1, 0, 0], [.25, .5, .25, 0], [0, .5, 0, .5], [0, 0, 1, 0]])
np.linalg.matrix_power(TM, 4)

array([[0.15625 , 0.5     , 0.28125 , 0.0625  ],
       [0.125   , 0.546875, 0.1875  , 0.140625],
       [0.140625, 0.375   , 0.453125, 0.03125 ],
       [0.0625  , 0.5625  , 0.0625  , 0.3125  ]])

In [6]:
# Exercise 3
initial_distribution = np.array([.25, .25, .25, .25])
transition_matrix = np.array([[0, 1, 0, 0], [.25, .5, .25, 0], [0, .5, 0, .5], [0, 0, 1, 0]])
chain_length = 100
n_trajectories = 2000

print(simulated_trajectories(initial_distribution, transition_matrix, chain_length, n_trajectories))
print(theoretical_statistics(initial_distribution, transition_matrix))

('Simulated average time in each state: ', [0.12483362279256412, 0.49582139623257676, 0.2517107782742292, 0.1276342027006299], 'Simulated average time before absorption: ', 100.0)
('Chain is regular and its stationary distribution is', [0.12500000000000008, 0.5000000000000002, 0.2499999999999997, 0.1249999999999999])


4. The invariant distribution, or the average time in each state, as seen in the output above is $\pi = \begin{pmatrix} 0.12550160563282714 & 0.4960341611372532 & 0.2515029613905918 & 0.12696127183932787 \\\end{pmatrix}$ or $\pi = \begin{pmatrix} \frac{1}{8} & \frac{1}{2} &\frac{1}{4} & \frac{1}{8} \\\end{pmatrix}.$ The simulated values are very close.

6. The expected first return time in state 2 is $\frac{1}{\pi_2} = \frac{1}{\frac{1}{2}} = 2.$